In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

Читаем входные файлы с данными

In [2]:
transactions = pd.read_csv('transactions.csv')

Примерный вид данных, содержащих транзакции клиентов

In [3]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,7391977,0 10:23:26,4814,1030,-2245.92,NaN
1,7391977,1 10:19:29,6011,7010,56147.89,NaN
2,7391977,1 10:20:56,4829,2330,-56147.89,NaN
3,7391977,1 10:39:54,5499,1010,-1392.47,NaN
4,7391977,2 15:33:42,5499,1010,-920.83,NaN


Для обучения будем использовать только траты клиентов

In [4]:
train_transactions = transactions[transactions.amount < 0].copy()

Добавим признак обозначающий день транзакции

In [5]:
train_transactions['day'] = train_transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

Так как нам тестовая выборка не дана, то мы сгенерируем её сами. Для этого возьмём все mcc_code представленные в обучающей выборке и предскажем их на 30 дней вперед.

In [6]:
test_transactions = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + train_transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']

Аналогично создадим сетку для обучения

In [7]:
train_grid = pd.DataFrame(columns=train_transactions.mcc_code.unique(), 
                          index=train_transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

Сделаем дополнительные признаки, касающиеся даты

In [8]:
for tr_table in [train_transactions, test_transactions, train_grid]:
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = tr_table['day'] % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30

Смержим наши данные для обучения с сеткой для обучения, группирую данные по дате, суммируя объём транзакций

In [9]:
train_transactions = \
    pd.merge(train_grid,
             train_transactions.groupby(['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code'])[['amount']]\
                 .sum().reset_index(),
             how='left').fillna(0)

Добавим признаки, которые показывают какой объём транзакций был месяц назад плюс минус один день.

In [10]:
for day_shift in [-1, 0, 1]:
    for month_shift in train_transactions.month_num.unique()[1:]:
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['amount'] + 1)
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 'amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                     on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)

Также добавим OHE кодирование для категорильного признака mcc_code

In [11]:
train = pd.get_dummies(train_transactions, columns=['mcc_code'])
test = pd.get_dummies(test_transactions, columns=['mcc_code'])

В качестве модели возьмём линейную регрессию, а предсказывать будет логарифм объёма, так как это будет лучше работать для метрики RMSLE.

In [12]:
c = train.columns.difference(['amount'])

In [13]:
def rmse(y_pred_log, y_test_log):
    return np.sqrt(np.mean((y_pred_log - y_test_log)**2))

In [14]:
days_total = len(sorted(train['day'].unique()))
val_days = 30

train_ = train[train.day < (days_total - val_days)]
validation = train[train.day >= (days_total - val_days)]

y_val_log = np.log1p(-validation['amount'])

In [15]:
clf = LinearRegression()
clf.fit(train_[c], np.log1p(-train_['amount']))
y_pred_log = clf.predict(validation[c])

rmse(y_pred_log, y_val_log)

27486460084.176357

Заменил линейную регрессию на XGBRegressor, public score вырос до 3.62648

In [16]:
import xgboost

clf = xgboost.XGBRegressor()
clf.fit(train_[c], np.log1p(-train_['amount']))
y_pred_log = clf.predict(validation[c])

rmse(y_pred_log, y_val_log)

3.5259127149013323

Обучил XGBRegressor с `n_estimators=200`, и RandomForestRegressor с `n_estimators=200`, взял ответ как 0.7\*xgb + 0.3\*rf, public score вырос до 3.58757

In [17]:
from sklearn.ensemble import RandomForestRegressor

clf = xgboost.XGBRegressor(n_estimators=200)
clf.fit(train_[c], np.log1p(-train_['amount']))

rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
rf.fit(train_[c], np.log1p(-train_['amount']))

pred1 = clf.predict(validation[c])
pred2 = rf.predict(validation[c])
y_pred_log = 0.7*pred1 + 0.3*pred2

rmse(y_pred_log, y_val_log)

3.4913789241298083

In [18]:
clf = xgboost.XGBRegressor(n_estimators=200)
clf.fit(train[c], np.log1p(-train['amount']))

rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
rf.fit(train[c], np.log1p(-train['amount']))

pred1 = clf.predict(test[c])
pred2 = rf.predict(test[c])
y_pred_log = 0.7*pred1 + 0.3*pred2

Предсказываем объём для тестовой выборки

In [19]:
test_transactions['volume'] = np.expm1(y_pred_log)

В качестве id нужно использовать 'mcc_code-day'

In [20]:
test_transactions['id'] = test_transactions[['mcc_code', 'day']].apply(lambda x: '-'.join(map(str, x)), axis=1)

In [21]:
test_transactions[['id', 'volume']].to_csv('baseline.csv', index=False)